<a href="https://colab.research.google.com/github/pitGitHub2016/PyPhD_Github/blob/master/DailyDataHandler_Base44_Betmechs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')

In [ ]:

import os
import io
import re
import zipfile
import requests
import pandas as pd
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse

# =========================
# 1) Fetch ZIP links
# =========================
def fetch_zip_links(page_url: str) -> list[str]:
    session = requests.Session()
    session.headers.update({"User-Agent": "Mozilla/5.0"})
    r = session.get(page_url, timeout=30)
    r.raise_for_status()
    soup = BeautifulSoup(r.text, "html.parser")
    links = []
    for a in soup.find_all("a", href=True):
        href = a["href"].strip()
        if ".zip" in href.lower():
            links.append(urljoin(page_url, href))
    return sorted(set(links))


# =========================
# 2) Download ZIPs uniquely

def _season_code_to_year(code: str) -> int:
    """Convert season code like '9394', '2324' to start year (int)."""
    if not code or not code.isdigit() or len(code) != 4:
        return -1
    try:
        first_two = int(code[:2])
        # Heuristic: if >= 90 → 1900s, else → 2000s
        if first_two >= 90:
            return 1900 + first_two
        else:
            return 2000 + first_two
    except Exception:
        return -1


def download_zips(zip_urls, zip_dir="football_zips"):
    """
    Downloads zip files uniquely into zip_dir.
    - Deletes the latest season's zip for each league before re-downloading.
    - Renames generic "data.zip" into {league}_{season}.zip.
    - Infers league code and season robustly from the URL.
    """
    os.makedirs(zip_dir, exist_ok=True)
    session = requests.Session()
    session.headers.update({"User-Agent": "Mozilla/5.0"})
    saved_files = []

    league_groups = {}

    for url in zip_urls:
        raw_fname = os.path.basename(urlparse(url).path)

        # Try to parse season + league from URL
        parts = url.split("/")
        season_part, league_part = None, None

        if len(parts) >= 3:
            season_candidate = parts[-2]
            if season_candidate.isdigit():
                season_part = season_candidate

        if parts and parts[-1]:
            league_candidate = parts[-1].split(".")[0].upper()
            if re.match(r"^[A-Z0-9]{1,4}$", league_candidate):
                league_part = league_candidate

        if not league_part:
            league_part = "GENERIC"
        if not season_part:
            season_part = "LATEST"

        fname = f"{league_part}_{season_part}.zip"
        league_groups.setdefault(league_part, []).append((url, fname, season_part))

    # --- Process each league group ---
    for league, files in league_groups.items():
        # Find latest season by converted year
        season_info = [(f[2], _season_code_to_year(f[2])) for f in files if f[2].isdigit()]
        if season_info:
            latest_code, latest_year = max(season_info, key=lambda x: x[1])
            latest_url, latest_fname, _ = next(f for f in files if f[2] == latest_code)
            latest_path = os.path.join(zip_dir, latest_fname)

            if os.path.exists(latest_path):
                print(f"🗑️ Deleting cached {league} {latest_code} ({latest_year}) → {latest_fname}")
                os.remove(latest_path)

            print(f"🔄 Latest season for {league} identified as {latest_code} ({latest_year}). Will re-fetch.")

        # Download files
        for url, fname, _ in files:
            out_path = os.path.join(zip_dir, fname)
            if os.path.exists(out_path):
                print(f"✅ {fname} already exists, skipping.")
                saved_files.append(out_path)
                continue

            print(f"⬇️  Downloading {url}")
            r = session.get(url, timeout=120)
            r.raise_for_status()
            with open(out_path, "wb") as f:
                f.write(r.content)
            saved_files.append(out_path)
            print(f"💾 Saved zip: {out_path}")

    return saved_files


# =========================
# 3) Column normalization
# =========================
def _normalize_columns(df: pd.DataFrame) -> pd.DataFrame:
    cleaned = []
    for c in df.columns:
        cc = str(c).replace("\ufeff", "").strip()
        cleaned.append(cc)
    df.columns = cleaned

    lower_map = {c.lower(): c for c in df.columns}
    if "div" in lower_map:
        df.rename(columns={lower_map["div"]: "Div"}, inplace=True)
    lower_map = {c.lower(): c for c in df.columns}
    if "date" in lower_map and lower_map["date"] != "Date":
        df.rename(columns={lower_map["date"]: "Date"}, inplace=True)
    return df


# =========================
# 4) Robust CSV reader
# =========================
def _read_csv_from_zip(zf: zipfile.ZipFile, member: str) -> pd.DataFrame:
    data = zf.read(member)
    for enc in ["utf-8", "latin-1"]:
        for sep in [None, ",", ";"]:
            try:
                df = pd.read_csv(io.BytesIO(data), encoding=enc, sep=sep, engine="python", on_bad_lines="skip")
                if not df.empty:
                    return _normalize_columns(df)
            except Exception:
                continue
    return pd.DataFrame()


# =========================
# 5) Date normalization
# =========================
def normalize_date_column(df: pd.DataFrame) -> pd.DataFrame:
    if "Date" not in df.columns:
        return df

    if "DateRaw" not in df.columns:
        df.rename(columns={"Date": "DateRaw"}, inplace=True)
    else:
        df["DateRaw"] = df["Date"].astype(str)

    date_raw_series = df["DateRaw"].astype(str).str.strip()
    out_year, out_month, out_day = [], [], []
    norm_date_strings = []

    for raw in date_raw_series:
        raw = raw.strip()
        day = month = year = None
        parts = raw.split("/") if "/" in raw else (raw.split("-") if "-" in raw else [])

        if len(parts) == 3:
            a, b, c = [p.strip() for p in parts]
            if len(a) == 4 and a.isdigit():
                year, month, day = a, b, c
            else:
                day, month, year = a, b, c

            def to_int(x):
                try:
                    return int(x)
                except Exception:
                    return None

            di, mi, yi = to_int(day), to_int(month), to_int(year)
            if yi is not None and len(year) == 2:
                yi = 1900 + yi if yi > 30 else 2000 + yi

            def in_range(x, lo, hi):
                return x is not None and lo <= x <= hi

            if not in_range(mi, 1, 12) and in_range(di, 1, 12):
                di, mi = mi, di

            if in_range(di, 1, 31) and in_range(mi, 1, 12) and yi is not None:
                dd, mm, yyyy = f"{di:02d}", f"{mi:02d}", f"{yi:04d}"
                out_day.append(dd); out_month.append(mm); out_year.append(yyyy)
                norm_date_strings.append(f"{yyyy}-{mm}-{dd}")
            else:
                out_day.append(None); out_month.append(None); out_year.append(None); norm_date_strings.append(None)
        else:
            out_day.append(None); out_month.append(None); out_year.append(None); norm_date_strings.append(None)

    df["DateDay"], df["DateMonth"], df["DateYear"] = out_day, out_month, out_year
    df["Date"] = pd.to_datetime(norm_date_strings, errors="coerce")
    return df


# =========================
# 6) Filtering rules
# =========================
def _find_existing_cols_case_insensitive(df: pd.DataFrame, wanted: list[str]) -> list[str]:
    wanted_lower = {w.lower(): w for w in wanted}
    actual = []
    for c in df.columns:
        if c.lower() in wanted_lower:
            actual.append(c)
    return actual

def filter_by_results_and_odds(df: pd.DataFrame) -> pd.DataFrame:
    results_wanted = ["FTHG", "FTAG", "FTR"]
    result_cols_present = _find_existing_cols_case_insensitive(df, results_wanted)
    if result_cols_present:
        df = df.dropna(subset=result_cols_present, how="any")

    odds_prefixes = [
        "1XB", "B365", "BF", "BFD", "BFA", "BFDH", "BFDD", "BFDA",
        "BMGM", "BV", "BS", "BWH",
        "CL", "GB", "IW", "LB", "PS", "SO", "SB", "SJ", "SY", "VC", "WH",
        "Bb", "Max", "Avg", "BFE"
    ]
    odds_cols = [c for c in df.columns if any(c.upper().startswith(p) for p in odds_prefixes)]
    if odds_cols:
        df = df[~df[odds_cols].isna().all(axis=1)]
    return df


# =========================
# 7) Combine per-league + global
# =========================
def combine_all_leagues(zip_dir="football_zips", out_dir="combined_by_league", global_file="ALL_LEAGUES_combined.csv"):
    os.makedirs(out_dir, exist_ok=True)
    zip_files = [os.path.join(zip_dir, f) for f in os.listdir(zip_dir) if f.lower().endswith(".zip")]

    league_data, all_data = {}, []

    for zp in sorted(zip_files):
        print(f"\n📦 Processing {os.path.basename(zp)}")
        try:
            with zipfile.ZipFile(zp, "r") as zf:
                members = [m for m in zf.namelist() if m.lower().endswith(".csv")]
                if not members:
                    print("  ⚠️ No CSVs inside zip."); continue

                for m in members:
                    df = _read_csv_from_zip(zf, m)
                    if df.empty:
                        print(f"  ⚠️ {m} yielded 0 rows."); continue

                    if "Div" not in df.columns:
                        df["Div"] = pd.NA
                    df["Div"] = df["Div"].ffill().bfill()

                    df = normalize_date_column(df)
                    df = filter_by_results_and_odds(df)

                    for league_code, grp in df.groupby("Div", dropna=True):
                        league_code = str(league_code).strip().upper()
                        if not league_code or league_code == "NAN":
                            continue
                        league_data.setdefault(league_code, []).append(grp)
                        all_data.append(grp)
        except zipfile.BadZipFile:
            print(f"  ❌ Bad zip: {zp}, skipping."); continue

    for league_code, dfs in league_data.items():
        combined_df = pd.concat(dfs, ignore_index=True, sort=False)
        combined_df["Div"] = combined_df["Div"].ffill().bfill().fillna(league_code)
        if "Date" in combined_df.columns:
            combined_df = combined_df.sort_values(by="Date", ascending=True, na_position="last")

        out_path = os.path.join(out_dir, f"{league_code}__combined.csv")
        short_out_path = os.path.join(out_dir, f"{league_code}_short_combined.csv")

        combined_df = combined_df[combined_df['DateYear'].astype(float) >= 2005]
        combined_df = combined_df[[x for x in combined_df.columns if "Unnamed:" not in x]]

        combined_df.to_csv(out_path, index=False, encoding="utf-8")
        combined_df.tail(50).to_csv(short_out_path, index=False, encoding="utf-8")
        print(f"✅ Saved {out_path} (rows: {len(combined_df)})")

    if all_data:
        global_df = pd.concat(all_data, ignore_index=True, sort=False)
        if "Date" in global_df.columns:
            global_df = global_df.sort_values(by=["Date", "Div"], ascending=[True, True], na_position="last")
        global_path = os.path.join(out_dir, global_file)
        global_df.to_csv(global_path, index=False, encoding="utf-8")
        print(f"\n🌍 Global file saved: {global_path} (rows: {len(global_df)})")


# =========================
# 8) Master run
# =========================
def build_all_leagues():
    print("🔎 Fetching zip links...")
    main_links = fetch_zip_links("https://www.football-data.co.uk/downloadm.php")
    new_links  = fetch_zip_links("https://www.football-data.co.uk/all_new_data.php")
    zip_urls = sorted(set(main_links + new_links))
    print(f"✅ Total zip links: {len(zip_urls)}")

    print("\n💾 Downloading zips...")
    download_zips(zip_urls, zip_dir="football_zips")

    print("\n🧱 Combining and normalizing...")
    combine_all_leagues(zip_dir="football_zips", out_dir="combined_by_league")

    print("\n🎉 Done. Files saved in 'combined_by_league/'")


if __name__ == "__main__":
    build_all_leagues()

🔎 Fetching zip links...
✅ Total zip links: 33

💾 Downloading zips...
🔄 Latest season for DATA identified as 2526 (2025). Will re-fetch.
⬇️  Downloading https://www.football-data.co.uk/mmz4281/0001/data.zip
💾 Saved zip: football_zips/DATA_0001.zip
⬇️  Downloading https://www.football-data.co.uk/mmz4281/0102/data.zip
💾 Saved zip: football_zips/DATA_0102.zip
⬇️  Downloading https://www.football-data.co.uk/mmz4281/0203/data.zip
💾 Saved zip: football_zips/DATA_0203.zip
⬇️  Downloading https://www.football-data.co.uk/mmz4281/0304/data.zip
💾 Saved zip: football_zips/DATA_0304.zip
⬇️  Downloading https://www.football-data.co.uk/mmz4281/0405/data.zip
💾 Saved zip: football_zips/DATA_0405.zip
⬇️  Downloading https://www.football-data.co.uk/mmz4281/0506/data.zip
💾 Saved zip: football_zips/DATA_0506.zip
⬇️  Downloading https://www.football-data.co.uk/mmz4281/0607/data.zip
💾 Saved zip: football_zips/DATA_0607.zip
⬇️  Downloading https://www.football-data.co.uk/mmz4281/0708/data.zip
💾 Saved zip: foot

In [ ]:
import requests
import json
import sys
import pandas as pd

# --- ⚙️ CONFIGURATION ---
APP_HOSTNAME = "betmechs-ee8f45ee.base44.app"
BASE44_API_KEY = "09b8177c650048309207ea18b26b58fb"

# Available leagues for data fetching
AVAILABLE_LEAGUES = {
    "E0": "Premier League", "D1": "Bundesliga", "SP1": "La Liga", "I1": "Serie A", "F1": "Ligue 1",
    "E1": "Championship", "E2": "League 1", "E3": "League 2", "EC": "Conference",
    "SC0": "Scottish Premiership", "D2": "Bundesliga 2", "I2": "Serie B", "SP2": "Segunda Division",
    # Add more as needed
}

def call_backend_function(function_name, payload={}, timeout=60):
    """Helper to call backend functions."""
    url = f"https://{APP_HOSTNAME}/api/functions/{function_name}"
    headers = {
        "Authorization": f"Bearer {BASE44_API_KEY}",
        "Content-Type": "application/json"
    }

    print(f"📞 Calling '{function_name}' with payload: {payload}")

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=timeout)
        response.raise_for_status()

        response_data = response.json()
        print(f"✅ Success: {response_data.get('message', 'OK')}")
        return True, response_data.get('data', [])

    except requests.exceptions.RequestException as e:
        print(f"❌ ERROR in '{function_name}': {e}", file=sys.stderr)
        if hasattr(e, 'response') and e.response:
            try:
                error_details = e.response.json()
                print(f"   Response: {error_details}", file=sys.stderr)
            except json.JSONDecodeError:
                print(f"   Response (non-JSON): {e.response.text}", file=sys.stderr)
        return False, []

def fetch_recent_league_data(league_code, limit=100):
    """Fetch the most recent N records from a specific league's history table."""
    if league_code not in AVAILABLE_LEAGUES:
        print(f"❌ Invalid league code: {league_code}")
        return

    out_dir="latest_history_in_base44_backend"
    os.makedirs(out_dir, exist_ok=True)

    success, data = call_backend_function("fetchRecentHistory", {"leagueCode": league_code, "limit": limit})

    if success and data:
        print(f"📊 Successfully fetched {len(data)} records")
        df = pd.DataFrame(data)

        outPath = os.path.join(out_dir, f"{league_code}__latest.csv")
        df.to_csv(outPath, index=False, encoding="utf-8")

    else:
        print("No data returned or an error occurred.")

def interactive_fetch_menu():

    for code, name in AVAILABLE_LEAGUES.items():
        print(f"  {code}: {name}")
        try:
            fetch_recent_league_data(code, 100)
        except Exception as e:
            print("❌ Error ... ", e)

if __name__ == "__main__":
    interactive_fetch_menu()

  E0: Premier League
📞 Calling 'fetchRecentHistory' with payload: {'leagueCode': 'E0', 'limit': 100}
✅ Success: Successfully fetched 100 records from History_E0.
📊 Successfully fetched 100 records
  D1: Bundesliga
📞 Calling 'fetchRecentHistory' with payload: {'leagueCode': 'D1', 'limit': 100}
✅ Success: Successfully fetched 0 records from History_D1.
No data returned or an error occurred.
  SP1: La Liga
📞 Calling 'fetchRecentHistory' with payload: {'leagueCode': 'SP1', 'limit': 100}
✅ Success: Successfully fetched 0 records from History_SP1.
No data returned or an error occurred.
  I1: Serie A
📞 Calling 'fetchRecentHistory' with payload: {'leagueCode': 'I1', 'limit': 100}
✅ Success: Successfully fetched 0 records from History_I1.
No data returned or an error occurred.
  F1: Ligue 1
📞 Calling 'fetchRecentHistory' with payload: {'leagueCode': 'F1', 'limit': 100}
✅ Success: Successfully fetched 0 records from History_F1.
No data returned or an error occurred.
  E1: Championship
📞 Calling 

In [ ]:
import os
import pandas as pd
from datetime import datetime, timedelta

# Define the root directories as used in your original notebook
LATEST_DATA_DIR = 'latest_history_in_base44_backend'
SOURCE_DATA_DIR = 'combined_by_league'

def find_new_records_chronological(league_code: str, lookback_days: int = 14) -> pd.DataFrame:
    """
    Compares the latest combined CSV from football-data.co.uk with the last
    N records from the Base44 backend by using a chronological filter.
    This is an efficient approach for daily syncs.

    Args:
        league_code (str): The league code (e.g., 'E0' for Premier League).
        lookback_days (int): The number of days to look back from the most
                             recent date in the backend data for comparison.

    Returns:
        pd.DataFrame: A DataFrame containing only the new records, or an empty
                      DataFrame if no new records are found.
    """
    print(f"\n🔍 Searching for new records for league: {league_code} using chronological filtering.")

    # Step 1: Define file paths for the two datasets
    latest_backend_path = os.path.join(LATEST_DATA_DIR, f'{league_code}__latest.csv')
    combined_source_path = os.path.join(SOURCE_DATA_DIR, f'{league_code}__combined.csv')

    # Step 2: Check if both files exist before proceeding
    if not os.path.exists(latest_backend_path):
        print(f"❌ Error: Backend data file not found at '{latest_backend_path}'.")
        return pd.DataFrame()
    if not os.path.exists(combined_source_path):
        print(f"❌ Error: Source data file not found at '{combined_source_path}'.")
        return pd.DataFrame()

    # Step 3: Load the datasets into pandas DataFrames
    try:
        df_source = pd.read_csv(combined_source_path, encoding='utf-8')
        df_backend = pd.read_csv(latest_backend_path, encoding='utf-8')
    except Exception as e:
        print(f"❌ Error loading CSV files: {e}")
        return pd.DataFrame()

    print(f"✅ Loaded source file with {len(df_source)} records.")
    print(f"✅ Loaded backend file with {len(df_backend)} records.")

    # Step 4: Ensure consistent data types and create a unique key for comparison
    # We now use the Date, Teams, and Bet365 odds as a robust key
    key_columns = ['Date', 'HomeTeam', 'AwayTeam', 'B365H', 'B365D', 'B365A']

    # Handle missing odds columns gracefully
    for col in ['B365H', 'B365D', 'B365A']:
        if col not in df_source.columns:
            df_source[col] = ''
        if col not in df_backend.columns:
            df_backend[col] = ''

    # Convert 'Date' to a datetime object for filtering
    df_source['Date'] = pd.to_datetime(df_source['Date'], errors='coerce')
    df_backend['Date'] = pd.to_datetime(df_backend['Date'], errors='coerce')

    # Drop rows with invalid dates
    df_source.dropna(subset=['Date'], inplace=True)
    df_backend.dropna(subset=['Date'], inplace=True)

    # Step 5: Chronological Filtering
    # Find the latest date in the backend data to set the filter start point
    if df_backend.empty:
      print("Backend data is empty. Cannot perform chronological filter.")
      return pd.DataFrame()

    latest_source_date = df_source['Date'].max()
    latest_backend_date = df_backend['Date'].max()
    #filter_start_date = latest_source_date
    filter_start_date = latest_backend_date
    #filter_start_date = latest_backend_date - timedelta(days=lookback_days)
    print("latest_source_date = ", latest_source_date)
    print("latest_backend_date = ", latest_backend_date)
    print("filter_start_date = ", filter_start_date)

    df_source_filtered = df_source[df_source['Date'] >= latest_backend_date].copy()
    df_backend_filtered = df_backend[df_backend['Date'] >= latest_backend_date].copy()
    print("df_source_filtered")
    print(df_source_filtered)
    print("df_backend_filtered")
    print(df_backend_filtered)

    print(f"⏱️ Filtering both datasets from {latest_backend_date.strftime('%Y-%m-%d')}.")

    # Step 6: Create the unique key for comparison
    df_source_filtered['unique_key'] = df_source_filtered[key_columns].astype(str).agg('-'.join, axis=1)
    df_backend_filtered['unique_key'] = df_backend_filtered[key_columns].astype(str).agg('-'.join, axis=1)

    # Step 7: Identify new records using a left anti-join
    merged = df_source_filtered.merge(df_backend_filtered, on='unique_key', how='left', indicator=True)
    new_records_df = merged[merged['_merge'] == 'left_only'].copy()

    # Drop the temporary merge indicator and unique key columns
    new_records_df.drop(columns=['_merge', 'unique_key'], inplace=True)

    if not new_records_df.empty:
        print(f"✅ Found {len(new_records_df)} new records to upload.")
    else:
        print("🎉 No new records found. Your backend data is up to date!")

    new_records_df = new_records_df[[x for x in new_records_df.columns if "_x" in x]]
    new_records_df.columns = [x.split("_")[0] for x in new_records_df.columns]
    print(new_records_df.columns)
    new_records_df = new_records_df[df_source.columns]

    return new_records_df

# --- Demonstration of Usage ---
if __name__ == '__main__':
    league_code = "E1"
    new_data_to_upload = find_new_records_chronological(league_code)

    if not new_data_to_upload.empty:
        print("\nReady to upload the following new records:")
        print(new_data_to_upload)

        os.makedirs("new_data_to_upload", exist_ok=True)
        new_data_to_upload.to_csv("new_data_to_upload/"+league_code+"_latestRun.csv",index=False)
        print("Succesfully stored new data ... ready to upload !!!")
    else:
      new_data_to_upload.to_csv("new_data_to_upload/"+league_code+"_latestRun.csv",index=False)
      print("new_data_to_upload.empty!!!!")


🔍 Searching for new records for league: E1 using chronological filtering.


/tmp/ipython-input-3281635508.py:40: DtypeWarning: Columns (107) have mixed types. Specify dtype option on import or set low_memory=False.
  df_source = pd.read_csv(combined_source_path, encoding='utf-8')


✅ Loaded source file with 11245 records.
✅ Loaded backend file with 100 records.
latest_source_date =  2025-09-14 00:00:00
latest_backend_date =  2024-04-01 00:00:00
filter_start_date =  2024-04-01 00:00:00
df_source_filtered
      Div     DateRaw       HomeTeam        AwayTeam  FTHG  FTAG FTR  HTHG  \
10546  E1  01/04/2024        Ipswich     Southampton   3.0   2.0   H   1.0   
10547  E1  01/04/2024        Swansea             QPR   0.0   1.0   A   0.0   
10548  E1  01/04/2024     Sunderland       Blackburn   1.0   5.0   A   0.0   
10549  E1  01/04/2024          Stoke    Huddersfield   1.0   1.0   D   0.0   
10550  E1  01/04/2024  Middlesbrough  Sheffield Weds   2.0   0.0   H   1.0   
...    ..         ...            ...             ...   ...   ...  ..   ...   
11240  E1  13/09/2025      West Brom           Derby   0.0   1.0   A   0.0   
11241  E1  13/09/2025        Watford       Blackburn   0.0   1.0   A   0.0   
11242  E1  13/09/2025        Preston   Middlesbrough   2.0   2.0   D   1

In [ ]:
import requests
import json
import time
import sys
import csv
from datetime import datetime

# --- ⚙️ CONFIGURATION ---
APP_HOSTNAME = "betmechs-ee8f45ee.base44.app"
BASE44_API_KEY = "09b8177c650048309207ea18b26b58fb"
REQUEST_DELAY = 1.0  # 1 second delay between requests

# Available leagues for data upload
AVAILABLE_LEAGUES = {
    # England
    "E0": "Premier League", "E1": "Championship", "E2": "League 1", "E3": "League 2", "EC": "Conference",
    # Scotland
    "SC0": "Scottish Premiership", "SC1": "Scottish Championship", "SC2": "Scottish League 1", "SC3": "Scottish League 2",
    # Germany
    "D1": "Bundesliga", "D2": "Bundesliga 2",
    # Italy
    "I1": "Serie A", "I2": "Serie B",
    # Spain
    "SP1": "La Liga", "SP2": "Segunda Division",
    # France
    "F1": "Ligue 1", "F2": "Ligue 2",
    # Other leagues
    "N1": "Eredivisie", "B1": "Belgian Pro League", "P1": "Primeira Liga", "P2": "Liga de Honra",
    "T1": "Super Lig", "G1": "Super League Greece"
}

def call_backend_function(function_name, payload={}, timeout=300):
    """Helper to call backend functions with extended timeout for upload operations."""
    url = f"https://{APP_HOSTNAME}/api/functions/{function_name}"
    headers = {
        "Authorization": f"Bearer {BASE44_API_KEY}",
        "Content-Type": "application/json"
    }

    print(f"📞 Calling '{function_name}' for league {payload.get('leagueCode', 'unknown')}...")

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=timeout)
        response.raise_for_status()

        response_data = response.json()
        print(f"✅ Success: {response_data.get('message', 'OK')}")

        # Print detailed log if available
        if 'log' in response_data:
            for log_entry in response_data['log']:
                print(f"   📋 {log_entry}")

        # Add delay after each request to avoid rate limits
        time.sleep(REQUEST_DELAY)
        return True, response_data

    except requests.exceptions.RequestException as e:
        print(f"❌ ERROR in '{function_name}': {e}", file=sys.stderr)
        if hasattr(e, 'response') and e.response:
            try:
                error_details = e.response.json()
                print(f"   Response: {error_details}", file=sys.stderr)
            except json.JSONDecodeError:
                print(f"   Response (non-JSON): {e.response.text}", file=sys.stderr)
        time.sleep(REQUEST_DELAY)
        return False, None

def generate_sample_data(league_code, num_matches=10):
    """Generate sample match data for testing."""
    sample_matches = []
    teams = {
        "E0": ["Arsenal", "Chelsea", "Liverpool", "Man City", "Man United", "Tottenham"],
        "D1": ["Bayern Munich", "Dortmund", "RB Leipzig", "Bayer Leverkusen"],
        "SP1": ["Real Madrid", "Barcelona", "Atletico Madrid", "Sevilla"],
        "I1": ["Juventus", "AC Milan", "Inter", "Napoli"],
        "F1": ["PSG", "Lyon", "Marseille", "Monaco"]
    }

    league_teams = teams.get(league_code, ["Team A", "Team B", "Team C", "Team D"])

    for i in range(num_matches):
        home_team = league_teams[i % len(league_teams)]
        away_team = league_teams[(i + 1) % len(league_teams)]

        if home_team == away_team:
            away_team = league_teams[(i + 2) % len(league_teams)]

        match = {
            "Div": league_code,
            "Date": f"15/08/24",  # Use consistent date format
            "HomeTeam": home_team,
            "AwayTeam": away_team,
            "FTHG": i % 4,  # Home goals 0-3
            "FTAG": (i + 1) % 3,  # Away goals 0-2
            "FTR": "H" if (i % 4) > ((i + 1) % 3) else ("A" if (i % 4) < ((i + 1) % 3) else "D"),
            "B365H": round(1.5 + (i % 10) * 0.3, 2),  # Sample odds
            "B365D": round(3.0 + (i % 5) * 0.2, 2),
            "B365A": round(2.0 + (i % 8) * 0.4, 2),
            "Referee": f"Referee {i + 1}"
        }
        sample_matches.append(match)

    return sample_matches

def read_csv_file(file_path):
    """Read match data from a CSV file."""
    matches = []
    try:
        with open(file_path, 'r', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                # Convert empty strings to None
                processed_row = {}
                for key, value in row.items():
                    processed_row[key] = value if value.strip() else None
                matches.append(processed_row)

        print(f"📁 Successfully read {len(matches)} matches from {file_path}")
        return matches
    except FileNotFoundError:
        print(f"❌ File not found: {file_path}")
        return []
    except Exception as e:
        print(f"❌ Error reading CSV file: {e}")
        return []

def upload_data_to_league(league_code, match_data):
    """Upload match data to a specific league table. This function only appends data."""
    if league_code not in AVAILABLE_LEAGUES:
        print(f"❌ Invalid league code: {league_code}")
        print(f"Available leagues: {', '.join(AVAILABLE_LEAGUES.keys())}")
        return False

    if not match_data:
        print(f"❌ No match data provided for {league_code}")
        return False

    print(f"🚀 Starting upload for {AVAILABLE_LEAGUES[league_code]} ({league_code})")
    print(f"📊 Appending {len(match_data)} matches...")

    payload = {
        "leagueCode": league_code,
        "matchData": match_data,
    }

    success, result = call_backend_function("uploadHistoricalData", payload)

    if success and result:
        inserted_count = result.get('recordsInserted', 0)
        print(f"✅ Successfully appended {inserted_count} matches to {league_code}")
        return True
    else:
        print(f"❌ Failed to upload data to {league_code}")
        return False

def bulk_upload_sample_data():
    """Upload sample data to multiple leagues."""
    test_leagues = ["E0", "D1", "SP1", "I1", "F1"]

    print("🚀 Starting bulk upload of sample data...")
    print(f"Target leagues: {', '.join(test_leagues)}")
    print("⚠️  This will append 10 sample matches to each league")

    confirm = input("\nContinue? (yes/no): ")
    if confirm.lower() != 'yes':
        print("❌ Operation cancelled.")
        return

    success_count = 0
    for league_code in test_leagues:
        print(f"\n--- Processing {league_code} ({AVAILABLE_LEAGUES[league_code]}) ---")
        sample_data = generate_sample_data(league_code, 10)

        if upload_data_to_league(league_code, sample_data):
            success_count += 1
        else:
            print(f"❌ Failed to upload to {league_code}, continuing...")

    print(f"\n🎉 Bulk upload completed!")
    print(f"✅ Successfully uploaded to: {success_count}/{len(test_leagues)} leagues")

def interactive_data_upload():
    """Interactive menu for data upload operations."""
    while True:
        print("\n" + "="*60)
        print("📤 HISTORICAL DATA UPLOAD TOOL")
        print("="*60)
        print("Available options:")
        print("1. Upload sample data to single league")
        print("2. Upload sample data to multiple leagues (bulk test)")
        print("3. Upload data from CSV file")
        print("4. Show available leagues")
        print("5. Exit")

        choice = input("\nEnter your choice (1-5): ").strip()

        if choice == "1":
            print("\nAvailable leagues:")
            for code, name in AVAILABLE_LEAGUES.items():
                print(f"  {code}: {name}")

            league_code = input("\nEnter league code: ").strip().upper()
            if league_code in AVAILABLE_LEAGUES:
                num_matches = int(input("Number of sample matches (default 10): ") or "10")
                sample_data = generate_sample_data(league_code, num_matches)
                print(sample_data)
                time.sleep(3000)
                upload_data_to_league(league_code, sample_data)
            else:
                print("❌ Invalid league code.")

        elif choice == "2":
            bulk_upload_sample_data()

        elif choice == "3":
            file_path = input("Enter CSV file path: ").strip()
            league_code = input("Enter target league code: ").strip().upper()

            if league_code not in AVAILABLE_LEAGUES:
                print("❌ Invalid league code.")
                continue

            matches = read_csv_file(file_path)
            if matches:
                # This operation is now append-only. Existing data will not be cleared.
                upload_data_to_league(league_code, matches)

        elif choice == "4":
            print("\nAll available leagues:")
            for code, name in AVAILABLE_LEAGUES.items():
                print(f"  {code:4s}: {name}")

        elif choice == "5":
            print("👋 Goodbye!")
            break

        else:
            print("❌ Invalid choice. Please enter 1-5.")

if __name__ == "__main__":
    # You can run this in different ways:

    # Option 1: Interactive menu
    #interactive_data_upload()

    # Option 2: Upload sample data directly (will append)
    # sample_data = generate_sample_data("E0", 20)
    # upload_data_to_league("E0", sample_data)

    # Option 3: Upload from CSV file (will append)
    league_code = "E1"
    matches = read_csv_file("new_data_to_upload/"+league_code+"_latestRun.csv")
    #matches = read_csv_file("combined_by_league/"+league_code+"__combined.csv")

    if len(matches)>0:
        upload_data_to_league(league_code, matches)
    else:
      print("matches = empty ! No new records need to be uploaded!")

📁 Successfully read 695 matches from new_data_to_upload/E1_latestRun.csv
🚀 Starting upload for Championship (E1)
📊 Appending 695 matches...
📞 Calling 'uploadHistoricalData' for league E1...
✅ Success: Successfully appended 695 matches for E1
   📋 Starting data APPEND process for E1 (History_E1)
   📋 Received 695 matches to process
   📋 Processed and validated 695 matches
   📋 Appended batch 1/14 (50 records)
   📋 Appended batch 2/14 (50 records)
   📋 Appended batch 3/14 (50 records)
   📋 Appended batch 4/14 (50 records)
   📋 Appended batch 5/14 (50 records)
   📋 Appended batch 6/14 (50 records)
   📋 Appended batch 7/14 (50 records)
   📋 Appended batch 8/14 (50 records)
   📋 Appended batch 9/14 (50 records)
   📋 Appended batch 10/14 (50 records)
   📋 Appended batch 11/14 (50 records)
   📋 Appended batch 12/14 (50 records)
   📋 Appended batch 13/14 (50 records)
   📋 Appended batch 14/14 (45 records)
   📋 ✅ Successfully appended 695 matches for E1
✅ Successfully appended 695 matches to E1

In [ ]:
import requests
import json
import time
import sys

# --- ⚙️ CONFIGURATION ---
APP_HOSTNAME = "betmechs-ee8f45ee.base44.app"
BASE44_API_KEY = "09b8177c650048309207ea18b26b58fb"
BATCH_SIZE = 50
MAX_BATCHES = 200 # Safety limit to prevent infinite loops

# --- Pipeline Functions ---
LEAGUE_PNL_FUNCTIONS = {
    "E1": "calculateE1Pnl","E0": "calculateE0Pnl",
}
INDEX_CREATION_FUNCTIONS = [
    "createPremierLeagueIndexes", "createBundesligaIndexes", "createLaLigaIndexes",
    "createSerieAIndexes", "createLigue1Indexes",
]
PERFORMANCE_CALCULATION_FUNCTIONS = [
    "calculatePremierLeaguePerformance", "calculateBundesligaPerformance", "calculateLaLigaPerformance",
    "calculateSerieAPerformance", "createLigue1Performance",
]

def call_backend_function(function_name, payload={}, timeout=60):
    """Helper to call backend functions."""
    url = f"https://{APP_HOSTNAME}/api/functions/{function_name}"
    headers = {
        "Authorization": f"Bearer {BASE44_API_KEY}",
        "Content-Type": "application/json"
    }

    print(f"📞 Calling '{function_name}' with payload: {json.dumps(payload)}")

    try:
        response = requests.post(url, headers=headers, json=payload, timeout=timeout)
        response.raise_for_status()

        response_data = response.json()
        print(f"✅ {function_name} success: {response_data.get('message', 'OK')}")
        return True, response_data
    except requests.exceptions.RequestException as e:
        print(f"❌ ERROR in '{function_name}': {e}", file=sys.stderr)
        if hasattr(e, 'response') and e.response:
            try:
                error_details = e.response.json()
                print(f"   Response: {error_details}", file=sys.stderr)
            except json.JSONDecodeError:
                print(f"   Response (non-JSON): {e.response.text}", file=sys.stderr)
        return False, None

def run_full_pipeline():
    """Run the full data processing pipeline from P&L to performance."""
    start_time = time.time()
    print("🚀 Starting full data processing pipeline...\n")

    # --- STEP 1: Calculate P&L for each league using batch processing ---
    print("--- STEP 1: Calculating P&L for all leagues ---")
    for league_code, pnl_func in LEAGUE_PNL_FUNCTIONS.items():
        print(f"\n--- Processing League: {league_code} ---")

        # 1a. Clear existing P&L data
        #print(f"Clearing old P&L data for {league_code}...")
        #success, _ = call_backend_function("clearPnlHistory", {"leagueCode": league_code})
        #if not success:
        #    print(f"🚨 Failed to clear P&L for {league_code}. Aborting this league.", file=sys.stderr)
        #    continue

        # 1b. Process P&L in batches
        offset = 0
        for i in range(MAX_BATCHES):
            print(f"Processing batch {i+1} for {league_code} (offset: {offset})...")
            success, data = call_backend_function(pnl_func, {"offset": offset})

            if not success:
                print(f"🚨 Batch failed for {league_code}. Aborting this league.", file=sys.stderr)
                break

            processed_count = data.get("processedCount", 0)
            if processed_count == 0:
                print(f"🏁 Finished processing {league_code}.")
                break

            offset += processed_count
            time.sleep(0.5) # Small delay to avoid overwhelming the server
        else:
            print(f"⚠️ Warning: Reached max batches for {league_code}. May be incomplete.", file=sys.stderr)

    """
    # --- STEP 2: Create betting indexes ---
    print("\n--- STEP 2: Creating betting indexes ---")
    for func_name in INDEX_CREATION_FUNCTIONS:
        success, _ = call_backend_function(func_name, {})
        if not success:
            print(f"🚨 Failed on {func_name}. Continuing...", file=sys.stderr)
        time.sleep(1)

    # --- STEP 3: Calculate strategy performance ---
    print("\n--- STEP 3: Calculating strategy performance ---")
    for func_name in PERFORMANCE_CALCULATION_FUNCTIONS:
        success, _ = call_backend_function(func_name, {})
        if not success:
            print(f"🚨 Failed on {func_name}. Continuing...", file=sys.stderr)
        time.sleep(1)
     """

    end_time = time.time()
    print(f"\n🎉 Pipeline completed in {end_time - start_time:.1f} seconds!")

if __name__ == "__main__":
    run_full_pipeline()

🚀 Starting full data processing pipeline...

--- STEP 1: Calculating P&L for all leagues ---

--- Processing League: E1 ---
Processing batch 1 for E1 (offset: 0)...
📞 Calling 'calculateE1Pnl' with payload: {"offset": 0}
✅ calculateE1Pnl success: Processed 50 matches.
Processing batch 2 for E1 (offset: 50)...
📞 Calling 'calculateE1Pnl' with payload: {"offset": 50}
✅ calculateE1Pnl success: Processed 50 matches.
Processing batch 3 for E1 (offset: 100)...
📞 Calling 'calculateE1Pnl' with payload: {"offset": 100}
✅ calculateE1Pnl success: Processed 50 matches.
Processing batch 4 for E1 (offset: 150)...
📞 Calling 'calculateE1Pnl' with payload: {"offset": 150}
✅ calculateE1Pnl success: Processed 50 matches.
Processing batch 5 for E1 (offset: 200)...
📞 Calling 'calculateE1Pnl' with payload: {"offset": 200}
✅ calculateE1Pnl success: Processed 50 matches.
Processing batch 6 for E1 (offset: 250)...
📞 Calling 'calculateE1Pnl' with payload: {"offset": 250}
✅ calculateE1Pnl success: Processed 50 mat

⚠️ Warning: Reached max batches for E1. May be incomplete.



--- Processing League: E0 ---
Processing batch 1 for E0 (offset: 0)...
📞 Calling 'calculateE0Pnl' with payload: {"offset": 0}
✅ calculateE0Pnl success: Processed 50 matches.
Processing batch 2 for E0 (offset: 50)...
📞 Calling 'calculateE0Pnl' with payload: {"offset": 50}
✅ calculateE0Pnl success: Processed 50 matches.
Processing batch 3 for E0 (offset: 100)...
📞 Calling 'calculateE0Pnl' with payload: {"offset": 100}
✅ calculateE0Pnl success: Processed 50 matches.
Processing batch 4 for E0 (offset: 150)...
📞 Calling 'calculateE0Pnl' with payload: {"offset": 150}
✅ calculateE0Pnl success: Processed 50 matches.
Processing batch 5 for E0 (offset: 200)...
📞 Calling 'calculateE0Pnl' with payload: {"offset": 200}
✅ calculateE0Pnl success: Processed 50 matches.
Processing batch 6 for E0 (offset: 250)...
📞 Calling 'calculateE0Pnl' with payload: {"offset": 250}
✅ calculateE0Pnl success: Processed 50 matches.
Processing batch 7 for E0 (offset: 300)...
📞 Calling 'calculateE0Pnl' with payload: {"o